# Service Restoration Observatory

# Changes occuring in the provision of {measure}: {Start date} - {end date}

Below are various time-series graphs showing changes in total nationwide {measure} provision as well as a breakdown by region and other demographic variables.  The following graphs are available:

* [Total {measure} number](#total)
    * [Total {measure} number by child codes](#child-codes)
* [Total {measure} number by practice](#practice)
* [Breakdown by region](#region)
* [Breakdown by age band](#age)
* [Breakdown by sex](#sex)


#### Methods
Using OpenSAFELY-TPP, covering 40% of England's population, we have assessed coding activity related to {measure} in general practice from the beginning of {start date} until the end of {end date}. The codelist used can be found here at [OpenSAFELY Codelists](link to codelist).  For each month within the study period, we have calculated the rate at which the code was recorded per 1000 registered patients.

All analytical code and output is available for inspection at the [OpenSAFELY GitHub repository](link to repo).

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from IPython.display import HTML
from IPython.core.display import HTML as Center
from utilities import *

%matplotlib inline

Center(""" <style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style> """)

In [ ]:
# Load measures df
measures_df_total = pd.read_csv('../output/measure_1_total.csv')
measures_df_event_code = pd.read_csv('../output/measure_1_event_code.csv')
measures_df_practice = pd.read_csv('../output/measure_1_practice_only.csv')
measures_df_by_region = pd.read_csv('../output/measure_1_by_region.csv')
measures_df_by_age = pd.read_csv('../output/measure_1_by_age_band.csv')
measures_df_by_sex = pd.read_csv('../output/measure_1_by_sex.csv')

codelist = pd.read_csv('../codelists/opensafely-structured-medication-review-nhs-england.csv')
codelist.round(16)
measures_df_event_code.round(16)

# temporary fix for population not working in Measures
measures_df_total = measures_df_total.groupby(
    ['date'])['event_x', 'population'].sum().reset_index()
measures_df_total['value'] = measures_df_total['event_x'] /measures_df_total['population']



# Get measures

class Measure:
  def __init__(self, id, numerator, denominator, group_by):
    self.id = id
    self.numerator = numerator
    self.denominator = denominator
    self.group_by = group_by
    
    
measures = [
    Measure(
        id="1_total",
        numerator="event_x",
        denominator="population",
        group_by=None
    ),

    Measure(
        id="1_event_code",
        numerator="event_x",
        denominator="population",
        group_by=["practice", "event_x_event_code"]
    ),

    Measure(
        id="1_practice_only",
        numerator="event_x",
        denominator="population",
        group_by=["practice"]
    ),

    
]


# Convert date column to datetime and sort by date
to_datetime_sort(measures_df_total)
to_datetime_sort(measures_df_event_code)
to_datetime_sort(measures_df_practice)
to_datetime_sort(measures_df_by_region)
to_datetime_sort(measures_df_by_age)
to_datetime_sort(measures_df_by_sex)


# Redact small numbers from measures df
# redact_small_numbers(measures_df_total, 5, measures[0])

# Calculate rates

calculate_rate(measures_df_total, value_col='event_x', population_col='population', rate_per=1000)
calculate_rate(measures_df_event_code, value_col='event_x', population_col='population', rate_per=1000)
calculate_rate(measures_df_practice, value_col='event_x', population_col='population', rate_per=1000)
calculate_rate(measures_df_by_region, value_col='event_x', population_col='population', rate_per=1000)
calculate_rate(measures_df_by_age, value_col='event_x', population_col='population', rate_per=1000)
calculate_rate(measures_df_by_sex, value_col='event_x', population_col='population', rate_per=1000)


<a id='total'></a>
## Total {measure} Number

In [ ]:
plot_measures(measures_df_total, title='Total {Measure} across whole population', measure_id='measure_1', column_to_plot='rate', category=False, y_label='Rate per 1000')

<a id='child-codes'></a>
### Sub totals by child codes

Events for the top x subcodes across the study period

In [ ]:
child_table = create_child_table(df=measures_df_event_code, code_df=codelist, code_column='code', term_column='term', measure='event_x')
child_table

In [ ]:
plot_measures(measures_df_event_code, title='Total {Measure} across whole populatio for top 5 child codes', measure_id='measure_1', column_to_plot='rate', category='event_x_event_code', y_label='Rate per 1000')

<a id='practice'></a>
## Total number by practice

In [ ]:
practice_df = pd.read_csv('../output/input_practice_count.csv')
practices_dict =calculate_statistics_practices(measures_df_practice, practice_df,"2020-02-01")
print(f'Practices included entire period: {practices_dict["total"]["number"]} ({practices_dict["total"]["percent"]}%)')
print(f'Practices included within last year: {practices_dict["year"]["number"]} ({practices_dict["year"]["percent"]}%)')
print(f'Practices included within last 3 months: {practices_dict["months_3"]["number"]} ({practices_dict["months_3"]["percent"]}%)')



In [ ]:
interactive_deciles_chart(measures_df_practice, period_column='date', column='event_x', title='Decile chart',ylabel='rate per 1000')

<a id="region"></a>
## Breakdown by Region

Number of events within each group.

In [ ]:
measures_df_by_region['region'] = measures_df_by_region['region'].replace(np.nan, 'NA')
counts_df = calculate_statistics_demographics(df=measures_df_by_region, demographic_var='region', end_date="2021-02-01", event_column='event_x')
counts_df

In [ ]:
plot_measures(measures_df_by_region, title='Breakdown by region',measure_id='measure_1', column_to_plot='rate', category='region', y_label='Rate per 1000')

<a id="age"></a>
## Breakdown by Age

Number of events within each group.

In [ ]:
counts_df = calculate_statistics_demographics(df=measures_df_by_age, demographic_var='age_band', end_date="2021-02-01", event_column='event_x')
counts_df

In [ ]:
plot_measures(measures_df_by_age, title='Breakdown by age',measure_id='measure_1', column_to_plot='rate', category='age_band', y_label='Rate per 1000')

<a id="sex"></a>
## Breakdown by Sex

Number of events within each group.

In [ ]:
counts_df = calculate_statistics_demographics(df=measures_df_by_sex, demographic_var='sex', end_date="2021-02-01", event_column='event_x')
counts_df

In [ ]:
plot_measures(measures_df_by_sex, title='Breakdown by sex',measure_id='measure_1', column_to_plot='rate', category='sex', y_label='Rate per 1000')